In [10]:
import onnx
from onnx.shape_inference import infer_shapes
from onnx import helper
import os
print("ONNX version:", onnx.__version__)

ONNX version: 1.12.0


In [12]:
# Set some global variables
DESIRED_NODE = "Mul_"
ONNX_MODEL_FOLDER = "onnx_models"

In [13]:
# Load the baseline tinyyolo model
model = onnx.load("../tinyyolo_infershapes.onnx")
mulnodes = []
# Get the desired node 
for index,n in enumerate(model.graph.node):
    if DESIRED_NODE in n.name:
        mulnodes.append(index)

# Get the first node that is present in the graph
mul_node = model.graph.node[mulnodes[0]]
in_tensors = mul_node.input
out_tensor = mul_node.output
in_value_info = []
out_value_info = []

# Get the Valueprotos for the in and out tensors
for x in model.graph.value_info:
    if in_tensors[0] in x.name or in_tensors[1] in x.name:
        in_value_info.append(x)
    elif out_tensor[0] in x.name:
        out_value_info.append(x)

In [15]:
# Create a new graph from the desired node. 
graph = helper.make_graph(
    nodes=[mul_node],
    name="simple_graph",
    inputs=[in_value_info[0], in_value_info[1]],
    # initializer = [in_value_info[1]],
    outputs=[out_value_info[0]],
)

# Save the model to the onnx model folder
if not os.path.exists(ONNX_MODEL_FOLDER):
    os.makedirs(ONNX_MODEL_FOLDER)
onnx_model = helper.make_model(graph, producer_name="multiply-model")
onnx.save(onnx_model, ONNX_MODEL_FOLDER + '\multiply_model_split.onnx')

NameError: name 'onnx_model' is not defined

True